In [35]:
# Libs
import pandas as pd
import numpy as np
import os
import gc
import pymc as pm
import arviz as az
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

os.chdir('/Users/petepritchard/Documents/Projects/BigDataBowl')

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

plt.rcParams["figure.constrained_layout.use"] = True

print(f"Running on PyMC v{pm.__version__}")

Running on PyMC v5.19.1


In [23]:
def process_model_data(file_path):

    df = pd.read_csv(file_path)
    features = [
        'down', 
        'yardsToGo', 
        'absoluteYardlineNumber',
        'avg_def_speed',
        'std_def_acc', 
        'def_players_near_los', 
        'avg_def_spacing',
        'avg_olb_orientation',
        'avg_olb_motion_angle', 
        'avg_ilb_orientation',
        'avg_ilb_motion_angle', 
        'avg_mlb_orientation', 
        'avg_mlb_motion_angle',
        'avg_lb_orientation', 
        'avg_lb_motion_angle', 
        'avg_cb_orientation',
        'avg_cb_motion_angle', 
        'avg_ss_orientation', 
        'avg_ss_motion_angle',
        'avg_fs_orientation', 
        'avg_fs_motion_angle', 
    ]

    team_encoder = {team: idx for idx, team in enumerate(df['possessionTeam'].unique())}
    df['team_idx'] = df['possessionTeam'].map(team_encoder)

    X = df[features]
    y = df['successfulBlitz']

    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)

    return df, X_scaled, y, features, team_encoder

model_data, X, y, features, team_encoder = process_model_data('data/clean_data.csv')

In [ ]:
with pm.Model() as hierarchical_model:
    # Global params
    global_alpha  = pm.Normal('global_alpha', mu=0, sigma=10)
    global_beta = pm.Normal('global_beta', mu=0, sigma=1, shape=X.shape[1])
    # Global priors
    alpha_sigma = pm.HalfNormal('alpha_sigma', sigma=10)
    beta_sigma = pm.HalfNormal('beta_sigma', sigma=10, shape=X.shape[1])
    # Team level
    team_alpha = pm.Normal('team_alpha',
                           mu=global_beta,
                           sigma=alpha_sigma,
                           shape=len(model_data['team_idx'].unique()))
    team_beta = pm.Normal('team_beta',
                          mu=global_beta,
                          sigma=beta_sigma,
                          shape=(len(model_data['team_idx'].unique()), X.shape[1]))
    
    team_indices = model_data['team_idx'].values
    mu = team_alpha[team_indices] + pm.math.sum(X * team_beta[team_indices], axis=1)
    theta = pm.Deterministic('theta', pm.math.sigmoid(mu))
    y_obs = pm.Bernoulli('y_obs', p=theta, observed=y)

    trace = pm.sample(2000, tune=1000, return_inferencedata=True, 
                          random_seed=42)

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(1)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/minicond


You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_8_8mevsr


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/minicond


You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error__q7xcbpp


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(10)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/minicon

...

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(10)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/minicon


You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_yrv1acoq


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(0.0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/minico


You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_wjw3ts1y


ValueError: Incompatible Elemwise input shapes [(32,), (21,)]

In [34]:
model_data, X, y, features, team_encoder = process_model_data('data/clean_data.csv')

n_teams = len(model_data['team_idx'].unique())
team_idx = model_data['team_idx'].values

with pm.Model() as hierarchical_blitz_model:
        # Hyperpriors for global intercept and coefficients
        global_alpha = pm.Normal('global_alpha', mu=0, sigma=10)
        
        # Horseshoe prior for feature selection
        # Allows for sparse feature effects
        tau = pm.HalfCauchy('tau', beta=1)
        nu = pm.HalfCauchy('nu', beta=1, shape=X.shape[1])
        global_beta = pm.Normal('global_beta', 
                                mu=0, 
                                sigma=tau * nu, 
                                shape=X.shape[1])
        
        # Hierarchical variance for team-level effects
        alpha_sigma = pm.HalfNormal('alpha_sigma', sigma=10)
        beta_sigma = pm.HalfNormal('beta_sigma', sigma=10, shape=X.shape[1])
        
        # Team-level intercepts and coefficients
        team_alpha = pm.Normal('team_alpha', 
                                mu=global_alpha, 
                                sigma=alpha_sigma, 
                                shape=n_teams)
        team_beta = pm.Normal('team_beta', 
                               mu=global_beta, 
                               sigma=beta_sigma, 
                               shape=(n_teams, X.shape[1]))
        
        # Linear predictor with team-specific effects
        mu = pm.math.invlogit(
            team_alpha[team_idx] + 
            pm.math.sum(team_beta[team_idx] * X, axis=1)
        )
        
        # Likelihood (Bernoulli for binary outcome)
        y_obs = pm.Bernoulli('y_obs', logit_p=pm.math.log(mu / (1 - mu)), observed=y)
        
        # Sampling with diagonal mass matrix and adapt_delta for better sampling
        trace = pm.sample(2000, 
                          tune=1000, 
                          return_inferencedata=True, 
                          random_seed=42,
                          target_accept=0.95)

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(1)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/minicond


You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_okz_q9aw


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(0.0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/minico


You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_kc9mcf9z


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/minicond

...

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Subtensor{i}([32 21], 1)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/minic


You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_ro76kzk2


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Subtensor{i}([32 21], 0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/minic


You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_qs9mv1ws

You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_xzlwkbw_


CompileError: Compilation failed (return status=1):
/opt/miniconda3/envs/pymc_env/bin/clang++ -dynamiclib -g -Wno-c++11-narrowing -fno-exceptions -fno-unwind-tables -fno-asynchronous-unwind-tables -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -fPIC -undefined dynamic_lookup -I/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/numpy/core/include -I/opt/miniconda3/envs/pymc_env/include/python3.12 -I/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/link/c/c_code -L/opt/miniconda3/envs/pymc_env/lib -fvisibility=hidden -o /Users/petepritchard/.pytensor/compiledir_macOS-14.5-arm64-arm-64bit-arm-3.12.8-64/lazylinker_ext/lazylinker_ext.so /Users/petepritchard/.pytensor/compiledir_macOS-14.5-arm64-arm-64bit-arm-3.12.8-64/lazylinker_ext/mod.cpp
dyld[64652]: Symbol not found: __ZNK4tapi2v119LinkerInterfaceFile28getPlatformsAndMinDeploymentEv
  Referenced from: <565CE761-C1EB-37F5-9738-E1BCF6F2EC75> /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolch

In [8]:
def create_hierarchical_blitz_model(X, y, team_idx, n_teams):

    with pm.Model() as hierarchical_blitz_model:

        global_alpha = pm.Normal('global_alpha', mu=0, sigma=10)
        global_beta = pm.Normal('global_beta', mu=0, sigma=1, shape=X.shape[1])

        alpha_sigma = pm.HalfNormal('alpha_sigma', sigma=10)
        beta_sigma = pm.HalfNormal('beta_sigma', sigma=10, shape=X.shape[1])

        team_alpha = pm.Normal('team_alpha', 
                                mu=global_alpha, 
                                sigma=alpha_sigma, 
                                shape=n_teams)
        team_beta = pm.Normal('team_beta', 
                               mu=global_beta, 
                               sigma=beta_sigma, 
                               shape=(n_teams, X.shape[1]))
        
        mu = team_alpha[team_idx] + pm.math.dot(X, team_beta[team_idx].T)
        
        theta = pm.Deterministic('theta', pm.math.sigmoid(mu))
        y_obs = pm.Bernoulli('y_obs', p=theta, observed=y)
        
        trace = pm.sample(2000, tune=1000, return_inferencedata=True, 
                          random_seed=42)
    
    return hierarchical_blitz_model, trace

In [24]:
model, trace  = create_hierarchical_blitz_model(
    X,
    y,
    model_data['team_idx'],
    len(team_encoder)
)

ValueError: Size length is incompatible with batched dimensions of parameter 0 theta:
len(size) = 1, len(batched dims theta) = 2. Size must be None or have length >= 2

In [ ]:
# Additional Notes on Hierarchical Modeling
"""
Key Improvements in Hierarchical Model:
1. Accounts for team-specific variations
2. Borrows information across teams
3. Provides more nuanced insights into blitz success

Recommended Next Steps:
1. Ensure your CSV includes:
   - team
   - down
   - yards_to_go
   - field_position
   - blitz_success (binary: 1 for success, 0 for failure)

2. Install required libraries:
   pip install pymc arviz pandas numpy scikit-learn matplotlib

Potential Further Improvements:
- Add interaction terms between team and features
- Incorporate temporal effects (game sequence, season)
- Include more contextual features (quarterback, defensive formation)

Model Interpretation:
- Global parameters show overall blitz success trends
- Team-specific parameters reveal individual team variations
- Intra-class correlation indicates how much teams differ
"""

In [36]:
from numpy.random import default_rng

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
RANDOM_SEED = sum(map(ord, "Data Containers in PyMC"))
rng = default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

In [37]:
true_beta = 3
true_std = 5
n_obs = 100
x = rng.normal(size=n_obs)
y = rng.normal(loc=true_beta * x, scale=true_std, size=n_obs)

with pm.Model() as no_data_model:
    beta = pm.Normal("beta")
    mu = pm.Deterministic("mu", beta * x)
    sigma = pm.Exponential("sigma", 1)
    obs = pm.Normal("obs", mu=mu, sigma=sigma, observed=y)
    idata = pm.sample(random_seed=RANDOM_SEED)

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float32}(1)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/en


You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_yr09ny0m


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float32}(1)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/en


You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_vja_u0yd


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float32}(1)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/en

...

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float64}(0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/en


You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_35mk44ss


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float32}(1)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1913, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1085, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1166, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/en


You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_e60hvyp6

You can find the C code in this temporary file: /var/folders/5s/16rgxtsj5yg0q76p3s5s7_yh0000gn/T/pytensor_compilation_error_70mb3ems


CompileError: Compilation failed (return status=1):
/opt/miniconda3/envs/pymc_env/bin/clang++ -dynamiclib -g -Wno-c++11-narrowing -fno-exceptions -fno-unwind-tables -fno-asynchronous-unwind-tables -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -fPIC -undefined dynamic_lookup -I/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/numpy/core/include -I/opt/miniconda3/envs/pymc_env/include/python3.12 -I/opt/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pytensor/link/c/c_code -L/opt/miniconda3/envs/pymc_env/lib -fvisibility=hidden -o /Users/petepritchard/.pytensor/compiledir_macOS-14.5-arm64-arm-64bit-arm-3.12.8-64/lazylinker_ext/lazylinker_ext.so /Users/petepritchard/.pytensor/compiledir_macOS-14.5-arm64-arm-64bit-arm-3.12.8-64/lazylinker_ext/mod.cpp
dyld[87946]: Symbol not found: __ZNK4tapi2v119LinkerInterfaceFile28getPlatformsAndMinDeploymentEv
  Referenced from: <565CE761-C1EB-37F5-9738-E1BCF6F2EC75> /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolch